In [1]:
import pandas as pd 
import numpy as np 
import networkx as nx 
import json 
import matplotlib.pyplot as plt 
import seaborn as sns
from datetime import datetime
import tqdm

In [2]:
G = nx.read_graphml('../data/clean/byuser.graphml',node_type=int)
print(len(G),len(G.edges))

8258 61366


In [3]:
for node in list(G.nodes)[:3]: print(G.nodes[node])
for node in list(G.edges)[:3]: print(node,G.edges[node])

{'province': 'PI', 'covidMeasure': 0, 'latitude': 43.85180773789867, 'longitude': 10.506207}
{'province': 'LU', 'covidMeasure': 0, 'latitude': 43.85135, 'longitude': 10.506207}
{'province': 'FI', 'covidMeasure': 0, 'latitude': 43.76973, 'longitude': 11.26562}
(0, 15775, 0) {'start': 414, 'end': 414, 'user': 'RobertaC2372', 'days': 0, 'geodistance': 0.0004577378986709846}
(0, 15775, 1) {'start': 414, 'end': 414, 'user': 'ClaudioC138', 'days': 0, 'geodistance': 0.0004577378986709846}
(0, 15775, 2) {'start': 414, 'end': 414, 'user': 'juan pablo A 116887066', 'days': 0, 'geodistance': 0.0004577378986709846}


In [4]:
df = pd.read_csv('../data/raw/geotuscany.csv', index_col=0)
df.head()

,position,name,city,menu,link,score,N_reviews,price,Name,Province,...,latitude,longitude,cuisines,meals,specialDiets,positionlink,province,Link,provinceOK,longitud§e
0,2018,Osteria al Borgo,Lucca,False,/Restaurant_Review-g187898-d23137800-Reviews-O...,5.0,6,NaN,Osteria al Borgo,lucca,...,43.851808,10.506207,"Pizza, Tuscan, Central-Italian",NaN,"Vegetarian Friendly, Vegan Options, Gluten Fre...",j39_https://maps.google.com/maps?saddr=&daddr=...,pisa,/Restaurant_Review-g187898-d23137800-Reviews-O...,0,10.495633
1,551,Acino Bianco,Marina di Massa,False,/Restaurant_Review-g194814-d12843022-Reviews-A...,4.5,92,2.5,Acino Bianco,massacarrara,...,44.004616,10.115488,Italian,Dinner,"Vegetarian Friendly, Gluten Free Options",yEU_https://maps.google.com/maps?saddr=&daddr=...,lucca,/Restaurant_Review-g194814-d12843022-Reviews-A...,0,NaN
2,308,El Artista Ristorante Argentino,Massa,False,/Restaurant_Review-g660766-d17543924-Reviews-E...,4.5,78,2.5,El Artista Ristorante Argentino,massacarrara,...,44.041000,10.124396,"Barbecue, Argentinean, Steakhouse","Dinner, Lunch",NaN,U0P_https://maps.google.com/maps?saddr=&daddr=...,lucca,/Restaurant_Review-g660766-d17543924-Reviews-E...,0,NaN
3,1285,La Locanda di Bivigliano,Bivigliano,False,/Restaurant_Review-g1957776-d13948598-Reviews-...,NaN,28,NaN,La Locanda di Bivigliano,florence,...,43.893859,11.322421,"Italian, Mediterranean, Barbecue, Tuscan, Cent...",NaN,Vegetarian Friendly,rg1_https://maps.google.com/maps?saddr=&daddr=...,prato,/Restaurant_Review-g1957776-d13948598-Reviews-...,0,NaN
4,1499,Beach Bar @PuntAla,Punta Ala,False,/Restaurant_Review-g194871-d19315573-Reviews-B...,2.5,2,1.0,Beach Bar @PuntAla,grosseto,...,42.841410,10.775602,"Italian, Mediterranean","Lunch, Brunch, Drinks",NaN,32Q_https://maps.google.com/maps?saddr=&daddr=...,grosseto,/Restaurant_Review-g194871-d19315573-Reviews-B...,1,NaN


In [13]:
print(nx.dijkstra_path(G,'3','4'))
print(nx.dijkstra_path(G,'3','4',weight = 'geodistance'))
print(nx.dijkstra_path(G,'3','4',weight = 'days'))

['3', '1450', '618', '432', '10466', '10601', '12968', '4']
['3', '6530', '2429', '2016', '1832', '999', '12970', '12968', '4']
['3', '1450', '2285', '3848', '13834', '12970', '12968', '4']


In [18]:
o = nx.dijkstra_predecessor_and_distance(G, '3', weight="days")

In [37]:
for node in G.nodes: 
    if len(list(G.neighbors(node)))==0: print(node)

9532
8120
14622
8907
14865
13900
4878
10171
14763
4518
15727
8110
7961
5464
15567
15549
12524
15650
12048
15686
8236
13998
14880
4603
4658
3533
12303
11595
14152
4667
4662
13939
5551
15353
14126
8196
15323
15446
14138
8792
4609
3247
6119
9395
4277
3285
2732
3213
3510
15759
3356
2793
6183
12879
6403
7828
1474
13934
12286
10727
236
6571
7
13135
1130
1557
11041
5116
1940
6637
10310
11039
3803
4499
10254
4809
5814
12989
915
8835
8771
9619
726
13275
11000
1309
4753
13683
13342
13344
13851
5439
1193
14220
4409
8601
10
15704
11950
42
8073
6613
5674
267
1048
2996
9860
7737
9753
5117
5831
4776
11522
4284
2973
9299
3099
4279
234
1414
11002
9765
1806
12374
637
1852
6150
2953
7417
3611
3024
4935
3260
756
11784
855
1934
783
12413
11203
6107
6682
9304
8635
10828
3493
339
422
10004
6189
1699
2549
9364
6259
6341
7682
8035
10623
14835
10982
10204
6529
15691
111
8108
6142
2713
1361
6024
2124
2070
10323
11649
3128
440
10841
11573
12132
9784
1721
3426
6160
4174
1376
7459
3224
12338
12016
15464
5564
5194
7

In [32]:
def single_source dijkstra(g,source):
    
    
    dist = dict()
    Q = list()
    
   
    heappush(Q, source)
 
    
    # set initial distance from the source to `v` as INFINITY
    dist = {node : float('inf') for node in g.nodes}
    dist[source] = 0
    prev = {node : -1 for node in g.nodes}
    done = {node : False for node in g.nodes}
    done[source] = True
    
    route = list()
    
    while Q:
 
        u = heappop(Q)      # Remove and return the best vertex
        
 
        # do for each neighbor `v` of `u`
        for v in g.neighbors(u):
            
            weight = 1
 
            # Relaxation step
            if not done[v] and (dist[u] + weight) < dist[v]:
                dist[v] = dist[u] + weight
                prev[v] = u
                heappush(Q, v)
 
        # mark vertex `u` as done so it will not get picked up again
        done[u] = True
    
    for i in g.nodes:
        if i != source and dist[i] <float('inf'):
            get_route(prev, i, route)
            print(f"Path ({source} —> {i}): Minimum cost = {dist[i]}, Route = {route}")
            route.clear()

# Time Respecting Path 

In [8]:
from heapq import heappop, heappush

### SHORTEST PATHS

In [160]:
def single_source_time_respecting_paths(g,source):
    
    def mintime(g, u, v, date = 0):
        k = 0
        mink = 0
        minimal = float('inf') 
        while True: 
            try :
                end = g.edges[(u,v,k)]['end']
                start = g.edges[(u,v,k)]['start']
            except: break
            if start >= date and end < minimal: 
                minimal = end 
                mink = k 
                
            k += 1
    
        return minimal, mink

    
    dist = dict()
    Q = list()
    
    heappush(Q, source)
 
    # set initial distance from the source to `v` as INFINITY
    dist = {node : float('inf') for node in g.nodes}
    dist[source] = 0
    
    time = {node : 0 for node in g.nodes}
    time[source] = 0
    
    prev = {node : -1 for node in g.nodes}
    index = {source: 0}
    done = {node : False for node in g.nodes}
    done[source] = True
    
    
    
    while Q:
 
        u = heappop(Q)      # Remove and return the best vertex
        
 
        # do for each neighbor `v` of `u`
        for v in g.neighbors(u):
            
            weight = 1
            firstvisit, firstk = mintime(g,u,v, date = time[u])
            # Relaxation step
            if not done[v] and (dist[u] + weight) < dist[v] and firstvisit < float('inf'):
                dist[v] = dist[u] + weight
                prev[v] = u
                time[v] = firstvisit
                index[v] = firstk
                heappush(Q, v)
 
        # mark vertex `u` as done so it will not get picked up again
        done[u] = True
        
        
    def get_route(prev, time, index, i, route):
        if i >= 0:
            get_route(prev, time, index, prev[i], route)
            route.append((i,index[i],time[i]))    
        
    route = list()
    routes = dict()
    for i in g.nodes:
        if i != source and dist[i] <float('inf'):
            get_route(prev, time, index, i, route)
            routes[(source,i)] = route.copy()
            #print(f"Path ({source} —> {i}): Minimum cost = {dist[i]}, Route = {route}")
            route.clear()
            
    return routes

In [161]:
r = single_source_time_respecting_paths(G,0)

In [162]:
def all_time_respecting_paths(g):
    all_routes = dict()
    for node in tqdm.tqdm(g.nodes):
        all_routes.update(single_source_time_respecting_paths(g,node))
    
    return all_routes


In [163]:
rout = all_time_respecting_paths(G)

  0%|▏                                                                               | 14/8258 [00:02<22:51,  6.01it/s]


KeyboardInterrupt: 

In [123]:
len(routes)

15972744

In [117]:
import pickle
with open('../data/clean/allTpaths.PICKLE','wb') as f:
    pickle.dump(routes,f)
    
        

In [117]:
import pickle
with open('../data/clean/allTpaths.PICKLE','rb') as f:
    routes = pickle.load(f)
    
        

In [131]:
def geoDistance(df,u,v):
    
    xi, yi = df.loc[u,['longitude','latitude']]
    xj, yj = df.loc[v,['longitude','latitude']]
    return ((xj-xi)**2 + (yj-yi)**2)**0.5

In [138]:
longestRoutes = list()
maxLength = 0
fastestRoutes = list()
maxVelocity = 0

for i, route in tqdm.tqdm(routes.items()) :
    
    length = len(route)
    
    j,k,_ = route[1]
    
    start = G.edges[(i[0],j,k)]['start']
    end = route[-1][-1]
    timeElapsed = end-start+1
    velocity = length / timeElapsed
    
    if velocity > maxVelocity:
        maxVelocity = velocity 
        fastestRoutes = [i]
        
    elif velocity == maxVelocity:
        fastestRoutes.append(i)
    
    
    if length > maxLength:
        maxLength = length 
        longestRoutes = [i]
        
    elif length == maxLength:
        longestRoutes.append(i)
    
    

100%|██████████████████████████████████████████████████████████████████| 15972744/15972744 [01:00<00:00, 264063.09it/s]


In [136]:
print(maxDistance, longestRoutes,'\n',maxVelocity,fastestRoutes)

59 [(15591, 11950)] 
 19.0 [(1886, 11883)]


In [137]:
routes[(1886, 11883)]

[(1886, 0, 0),
 (1943, 0, 243),
 (3136, 0, 243),
 (5844, 0, 243),
 (6610, 0, 243),
 (7499, 0, 243),
 (7945, 0, 243),
 (11206, 0, 243),
 (14799, 0, 243),
 (6612, 0, 244),
 (7087, 0, 244),
 (7955, 0, 244),
 (8886, 0, 244),
 (9972, 0, 244),
 (9987, 0, 244),
 (11287, 0, 244),
 (11374, 0, 244),
 (11413, 0, 244),
 (11883, 0, 244)]

In [21]:
float('inf') > float('inf')


False

### SHORTEST GEODISTANCE PATHS

In [9]:
def single_source_time_respecting_paths(g,source,weights=None):
    
    def mintime(g, u, v, date = 0):
        k = 0
        mink = 0
        minimal = float('inf') 
        while True: 
            try :
                end = g.edges[(u,v,k)]['end']
                start = g.edges[(u,v,k)]['start']
            except: break
            if start >= date and end < minimal: 
                minimal = end 
                mink = k 
                
            k += 1
    
        return minimal, mink

    
    dist = dict()
    Q = list()
    
    heappush(Q, source)
 
    # set initial distance from the source to `v` as INFINITY
    dist = {node : float('inf') for node in g.nodes}
    dist[source] = 0
    
    time = {node : 0 for node in g.nodes}
    time[source] = 0
    
    prev = {node : -1 for node in g.nodes}
    index = {source: 0}
    done = {node : False for node in g.nodes}
    done[source] = True
    
    
    
    while Q:
 
        u = heappop(Q)      # Remove and return the best vertex
        
 
        # do for each neighbor `v` of `u`
        for v in g.neighbors(u):
            
            firstvisit, firstk = mintime(g,u,v, date = time[u])
            
            # Relaxation step 
            if firstvisit == float('inf'): continue
                
            if weights ==None :weight = 1
            else: weight = g.edges[(u,v,firstk)][weights]
            
            if not done[v] and (dist[u] + weight) < dist[v]:
                
                dist[v] = dist[u] + weight
                prev[v] = u
                time[v] = firstvisit
                index[v] = firstk
                heappush(Q, v)
 
        # mark vertex `u` as done so it will not get picked up again
        done[u] = True
        
        
    def get_route(prev, time, index, i, route):
        if i >= 0:
            get_route(prev, time, index, prev[i], route)
            route.append((i,index[i],time[i]))    
        
    route = list()
    routes = dict()
    for i in g.nodes:
        if i != source and dist[i] <float('inf'):
            get_route(prev, time, index, i, route)
            routes[(source,i)] = route.copy()
            #print(f"Path ({source} —> {i}): Minimum cost = {dist[i]}, Route = {route}")
            route.clear()
            
    return routes

In [10]:
def all_time_respecting_paths(g,weights=None):
    all_routes = dict()
    for node in tqdm.tqdm(g.nodes):
        all_routes.update(single_source_time_respecting_paths(g,node,weights))
    
    return all_routes

In [11]:
Wroutes = all_time_respecting_paths(G,weights='geodistance')

100%|██████████████████████████████████████████████████████████████████████████████| 8258/8258 [20:16<00:00,  6.79it/s]


In [29]:
import pickle
with open('../data/clean/allTGWpaths.PICKLE','wb') as f:
    pickle.dump(Wroutes,f)
    

### FOREMOST PATHS